# Features of Julia: Random walk

An important model in science (biology, chemistry, physics, etc.) is Brownian motion, or random walk, a model of a molecule moving in a medium.

As a first way to explore Julia and some of its features, we'll code the simplest version: a particle jumping in 1D on the integers. 

**Exercises**:

**[1]** How does the `rand` function work? What options does it have?

In [ ]:
rand

**[2]** (i) Write a simple 1D random walk for a particle that starts at the origin and only visits the integers, jumping with probability $\frac{1}{2}$ one step to the left and $\frac{1}{2}$ one step to the right, in "global scope" (as you would in Matlab or Python).

(ii) Time it for 10^7 steps with

```@time begin 
    ... 
    end
```

**[3]** (i) Wrap the code in a function `walk`; it is easy now to give the function an argument which is the final time (to allow for code re-use). The function should return the final position.

(ii) But this is not the only reason for using functions in Julia: now time running the function, and time it again. What do you notice?

(iii) Give the function a **docstring**, i.e. a string written just *before* the function definition that describes what the function does. You can use multi-line strings using `"""`.

**[4]** Make a version of the function that stores the trajectory in a vector. In fact, there are (at least) two ways we could do so: pre-allocating the vector, or allowing it to grow. How do these compare in time?

**[5]** We could make a version that takes as second argument a flag, which tells us whether to calculate the trajectory or not. Does this affect the time?

## Visualization

We would like to plot the trajectory $x(n)$ as a function of the time step $n$. 

The `Plots.jl` meta-package provides a common syntax to several different plotting package "backends".

In [ ]:
# Pkg.add("Plots")    # install the Plots.jl package if you don't already have it -- do once in your installation

using Plots           # load the package in each session
gr()   # choose the GR backend

**[1]** Plot a single trajectory of the walker $x(n)$ as a function of $n$ using lines and squares. [The `plot` function takes a collection of $x$ coordinates and a collection of $y$ coordinates.] Add a horizontal dashed line at $x=0$.

**[2]** Switch to the PlotlyJS backend using `plotlyjs()` and plot it again. What advantage does this backend have?

**[3]** Draw a few walks using a loop.

### Interactive manipulation

If we want to explore the space of parameters `T` (maximum time of the walk) and `N` (number of walks), it would be nice to be able to manipulate them interactively.

This may be done using `Interact.jl`.

In [ ]:
# Pkg.add("Interact")

using Interact

**[4]** (i) `Interact.jl` provides a macro `@manipulate` that is written directly before a `for` loop.
Experiment with it:

In [ ]:
@manipulate for i in 1:10
    i
end

Use it to manipulate the number of walks.
The `for` loop should return the object that will be displayed, in this case the result of the `plot`.

(ii) Now use it to manipulate a double `for` loop, with the syntax

```
@manipulate for x in 1:X, y in 1:Y
  ...
end
```

You will want to remove the plot legend/key with `leg=false`, and to fix the limits of the plot with keywords `xlim` and `ylim` or commands `xlims!` and `ylims!`.

## Collecting data

Now that we have a feel for the behaviour of the system, let's collect some data in a few different ways, in order to calculate the variance of the positions of `N` walks after time `T`.

**[1]** (i) Write a function `walks` that calculates the variance of the final positions of `N` walks of length `T` each.

(ii) Use `@time` to time the function for small `N` and `T`. Note that it is not very useful.

#### BenchmarkTools.jl

**[2]** Instead, use the `BenchmarkTools.jl` package and the `@benchmark` or `@btime` macros.

### Investigating different coding styles

**[3]** First write a **generic** `walks` function that **takes a function as one of its arguments**.

**[4]** Let's use the `walks` function to compare the efficiency of different coding styles for our `walk` function:

(i) One that uses `rand(Bool)`.
    
(ii) One that uses Matlab(ish) vectorised syntax, using a vector of random numbers.

(iii) One using **array comprehensions**, i.e. expressions of the form

```
[x for i in 1:10]
```

(iv) One using **generator expressions**, i.e. expressions of the form
    
```
(x for i in 1:10)
``` 
    
What is the difference?

**[5]** How could you generate the random $-1$ and $+1$ directly, using a range? Is it a good idea?

**[6]** Use the function `bitrand`. How does it compare?

Of course, it's (presumably) *possible* to do the above in C++, but it's certainly much *harder*. And in Python it will be much *slower*.

**[7]** **Bonus question** Compare the above timings to your (former) favourite programming language. There are C and Python versions in the `random_walk_in_other_languages` directory. I will add any implementations you send me there!

## Data analysis

**[1]** Write a function to collect the final positions of `N` walks after time `T` in an array.

**[2]** Make an interactive visualization of a normalised histogram using the `histogram` function in `Plots.jl`. 

[3] Let's compare this to the known analytical result: a normal distribution with variance equal to the number of steps.

(i) Define the mean $\mu$ and standard deviation $\sigma$ using Unicode symbols, as `\mu<TAB>` and `\sigma<TAB>`.

(ii) Load the `Distributions.jl` package.

(iii) Compare the histogram with a transparent, filled PDF. Use the `pdf` function from `Distributions.jl`, and the `Normal` normal distribution object.

(iv) Save the figure to a PDF using `savefig`.

## Write and read data to and from disk

We wish to save the data we have calculated to a file on disk. The simplest solution is as follows.

**[1]** (i) Use `writedlm` ("write delimited") to save the data.

(ii) Use `readdlm` to load it back in.

Note that the format has changed. Although there are some options to `readdlm`, this is not a good solution to the problem.

The good solution is to use the `JLD2.jl` ("JuLia Data") package, which uses the standard HDF5 file format underneath. We will also save another variable for illustrative purposes.

In [ ]:
# Pkg.add("JLD2")

# install 
using JLD2

In [ ]:
i = 17

In [ ]:
@save "random_walk.jld" i

In [ ]:
@load "random_walk.jld"

In [ ]:
# save("random_walk.jld", "final_pos", data, "arbitrary", i)   # alternative

In [ ]:
# all = load("random_walk.jld")

The data is read in as a **dictionary**, associating to each variable name its value. We can extract these as

In [ ]:
j = all["arbitrary"]

Alternatively, we can *read only part of the data file*:

In [ ]:
final_pos = load("random_walk.jld", "final_pos");

Note that the type of the data has now been correctly preserved. This is the point of using `JLD`. 

# Workflow: Extract code into a Julia script

Since we have developed a quantity of code in this notebook, it is a good moment to extract the code into a Julia script file, with termination `.jl`, that we edit, e.g. with Juno.

We can then read in the file as if we typed it with

In [ ]:
include("walk.jl")

Note that Julia provides various commands for manipulating the filesystem:

In [ ]:
pwd()

In [ ]:
readdir()

Later on, we would make it into a `module` (separate namespace) and then into a Julia pacakge.

## Summary:

- Julia allows us to calculate **fast**


- **And** visualize, interactively, our data


- All in one place